In [76]:
import json

article_urls = json.load(open('test-data/article-urls.json'))
promise_titles = json.load(open('test-data/promise-titles.json'))

In [7]:
import requests
import re
def download_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    api_url = 'https://api.budgetwiser.org/api/news/get_by_url/'
    article_parsed = requests.get(api_url, {'url': url}).json()
    with open(fname, 'w') as f:
        json.dump(article_parsed, f)

for url in article_urls:
    download_article(url)

In [11]:
import os.path

def load_article(url):
    article_id = re.search('aid=(\d+)', url).group(1)
    fname = 'test-data/article-%s.json' % article_id
    if not os.path.isfile(fname):
        download_article(url)
    return json.load(open(fname))

In [13]:
import logging
from konlpy.tag import Twitter
twitter = Twitter()
stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 제공 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)
pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Alpha', 'Adjective')]

In [77]:
from gensim import similarities, models, corpora

dictionary_ko = corpora.Dictionary.load('ko.dict')
index = similarities.MatrixSimilarity.load('lsi.index')
lsi = models.LsiModel.load('lsi.model')

def find_similar(tagged_text, similarity_threshold=None):
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    vec_lsi = lsi[vec_bow]
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    return sorted(sims, key=lambda item: -item[1])


In [78]:
import csv

article_promise_manual = {}

# Load manually tagged article-promise matches
with open('test-data/article-promise-manual.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        article_title = row[0]
        _promise_titles = [t for t in row[1:] if t]
        if article_title:
            article_promise_manual[article_title] = _promise_titles

In [107]:
#print(article_promise_manual)

k = 10  # precision at k metric

k_values = []

for url in article_urls:
    article_parsed = load_article(url)
    # title + excerpt
    n_words = 60
    text_excerpt = " ".join(list(filter(lambda word: len(word)>1, article_parsed['text'].replace("\n", " ").split(" ")))[:n_words])
    search_text = article_parsed['title'] + " " + text_excerpt
    similar_promises = find_similar(pos(search_text))
    print("Article: %s" % article_parsed['title'])
    print(text_excerpt)
    print(pos(search_text))
    #print("  Top-5 algorithm matched promises:")
    #for rank, item in enumerate(similar_promises[:5]):
    #    print("  (Rank %d, %d%%) %s" % (rank, item[1]*100, promise_titles[item[0]]))
    
    print("  Manually tagged promises:")
    manual_promises = article_promise_manual[article_parsed['title']]
    if not manual_promises:
        print("  (None)")
        continue
    ranks = []
    for item in manual_promises:
        promise_idx = promise_titles.index(item)
        rank_score = (0, 0)
        for rank, sim in enumerate(similar_promises):
            if sim[0] == promise_idx:
                match = (rank, sim[1])
                break
        print("  (Rank %d, %d%%) %s" % (match[0], match[1]*100, item))
        ranks.append(match[0])
    precision_at_k = len(list(filter(lambda r: r < k, ranks))) / k
    print("Precision@%d = %.4f" % (k, precision_at_k))
    k_values.append(precision_at_k)
    print()

print("Average precision@%d: %.4f" % (k, sum(k_values)/len(k_values)))
    

Article: 국세청, 고액·상습 체납자 2만1403명 명단 공개...김우중 '369억원' 체납
【서울=뉴시스】 【서울=뉴시스】 【서울=뉴시스】 【서울=뉴시스】 고액 체납자 1위에 유지양 효자건설 회장...447억원 체납유병언 세모그룹 회장 자녀 증여서 115억원 납부안해재산 추적조사 통해 1조5752억원 세금 징수...193명 형사고발【세종=뉴시스】박상영 기자 국세청은 11일 2억원 이상 세금을 내지 않은 고액·상습 체납자 2만1403명의 명단을 공개했다.국세청은 국세 체납 이후 1년 넘게 내지 않고 버티고 있는 개인이나 법인의 명단을 매년 공개하고 있다. 올해는 기준금액이 체납 3억원에서 2억원 이상으로 낮아지면서 공개인원이 작년보다
['국세청/Noun', '고액/Noun', '상습/Noun', '체납/Noun', '만/Noun', '명/Noun', '명단/Noun', '공개/Noun', '김우중/Noun', '억원/Noun', '체납/Noun', '서울/Noun', '뉴시스/Noun', '서울/Noun', '뉴시스/Noun', '서울/Noun', '뉴시스/Noun', '서울/Noun', '뉴시스/Noun', '고액/Noun', '체납/Noun', '위/Noun', '유지/Noun', '효자/Noun', '건설/Noun', '회장/Noun', '억원/Noun', '체납/Noun', '유병언/Noun', '세모/Noun', '그룹/Noun', '회장/Noun', '자녀/Noun', '증여/Noun', '억원/Noun', '납부/Noun', '안해/Noun', '재산/Noun', '추적/Noun', '조사/Noun', '통해/Noun', '조/Noun', '억원/Noun', '세금/Noun', '징수/Noun', '명/Noun', '형사/Noun', '고발/Noun', '세종/Noun', '뉴시스/Noun', '박상영/Noun', '기자/Noun', '국세청/Noun', '일/Noun', '억원/Noun', '이상/Noun', '세금/Noun', '내지/No

Article: 신촌 대학가에 월세 10만원대 공공기숙사 생긴다
광흥창역 역세권 청년주택 조감도[서울시 제공] 광흥창역 역세권 청년주택 조감도[서울시 제공] 광흥창역 역세권 청년주택 활용(서울=연합뉴스) 박초롱 기자 연세대·이화여대·서강대·홍익대 대학들이 밀집한 신촌 역세권에 10만∼12만원에 입주할 있는 공공기숙사가 생긴다.서울시는 마포구 광흥창역 인근에 짓는 역세권 청년주택을 활용해 공공기숙사를 운영한다고 6일 밝혔다.이는 강서구 내발산동에 이어 서울에 번째로 생기는 공공기숙사다. 서울이 아닌 지방 출신 학생들이 입주할 있다.서울시는 광흥창역 역세권 청년주택 민간사업자인 이랜드와 강원도 삼척·인제·정선·철원, 경북 고령,
['신촌/Noun', '대학가/Noun', '월세/Noun', '만원/Noun', '공공/Noun', '기숙사/Noun', '생기다/Verb', '광흥창역/Noun', '역세/Noun', '청년/Noun', '주택/Noun', '조감도/Noun', '서울시/Noun', '광흥창역/Noun', '역세/Noun', '청년/Noun', '주택/Noun', '조감도/Noun', '서울시/Noun', '광흥창역/Noun', '역세/Noun', '청년/Noun', '주택/Noun', '활용/Noun', '서울/Noun', '연합뉴스/Noun', '박초롱/Noun', '기자/Noun', '연세대/Noun', '이화여대/Noun', '서강대/Noun', '홍익대/Noun', '대학/Noun', '밀집/Noun', '신촌/Noun', '역세/Noun', '만/Noun', '만원/Noun', '이다/Adjective', '하다/Verb', '있다/Adjective', '공공/Noun', '기숙사/Noun', '생기다/Verb', '서울시/Noun', '마포구/Noun', '광흥창역/Noun', '인근/Noun', '짓다/Verb', '역세/Noun', '청년/Noun', '주택/Noun', '활용/Noun', '하다/Verb', '공공/Noun', '기숙사

['동남아/Noun', '제주도/Noun', '싸다/Verb', '실제/Noun', '여행/Noun', '비용/Noun', '배/Noun', '사진/Noun', '픽사/Noun', '베이/Noun', '표/Noun', '컨슈머/Noun', '인사이트/Noun', '머니투데이/Noun', '김/Noun', '고금/Noun', '평/Noun', '기자/Noun', '세종대/Noun', '컨슈머/Noun', '인사이트/Noun', '개/Noun', '월간/Noun', '국내/Noun', '해외/Noun', '여행/Noun', '지출/Noun', '경비/Noun', '분석/Noun', '해외여행/Noun', '개별/Noun', '단체/Noun', '평균/Noun', '배/Noun', '이상/Noun', '비싸다/Adjective', '동남아시아/Noun', '여행/Noun', '제주도/Noun', '여행/Noun', '싸다/Verb', '통념/Noun', '뒤지다/Verb', '조사/Noun', '나오다/Verb', '세종대/Noun', '관광/Noun', '산업/Noun', '연구소/Noun', '여행/Noun', '전문/Noun', '리서치/Noun', '컨슈머/Noun', '인사이트/Noun', '지난/Noun', '월/Noun', '월/Noun', '숙박/Noun', '여행/Noun', '최근/Noun', '개월/Noun', '국내/Noun', '여행/Noun', '지난/Noun', '개월/Noun', '해외여행/Noun', '일정/Noun', '비용/Noun', '비교/Noun', '하다/Verb', '조사/Noun', '하다/Verb', '결과/Noun', '해외여행/Noun', '비용/Noun', '일/Noun', '평균/Noun', '국내/Noun', '여행/Noun', '비용/Noun', '배/Noun', '제주도/Noun', '경우/Noun', '배/Noun', '많다/Adjective', '것/Noun', '나타나다/Verb', '국내/Noun',

Article: “육아 해결할 길 없어 일 포기” 82년생 김지영들의 좌절
통계·증언으로 ‘82년생 김지영들’의 *이미지를 클릭하면 확대됩니다. *이미지를 클릭하면 확대됩니다. *이미지를 클릭하면 확대됩니다. *이미지를 클릭하면 확대됩니다. <82년생 김지영>(민음사·2016) 30대 여성이 경험하는 일상적 차별과 구조적 불평등을 사실적으로 그려낸 조남주 작가의 소설. 슬하에 딸을 서른네살 전업주부 김지영씨는 어느 갑자기 친정어머니 등으로 빙의하는 증세를 보인다. 남편이 주선한 정신 상담에서 김지영씨는 자신의 삶을 이야기한다. 소설은 김지영씨의 이야기를 들은 담당 의사가 그의 인생을 재구성해 기록한 리포트
['육아/Noun', '해결/Noun', '하다/Verb', '길/Noun', '없다/Adjective', '일/Noun', '포기/Noun', '년생/Noun', '김지영/Noun', '좌절/Noun', '통계/Noun', '증언/Noun', '년생/Noun', '김지영/Noun', '의/Noun', '이미지/Noun', '클릭/Noun', '하다/Verb', '되다/Verb', '이미지/Noun', '클릭/Noun', '하다/Verb', '되다/Verb', '이미지/Noun', '클릭/Noun', '하다/Verb', '되다/Verb', '이미지/Noun', '클릭/Noun', '하다/Verb', '되다/Verb', '년생/Noun', '김지영/Noun', '민음사/Noun', '대다/Verb', '여성/Noun', '경험/Noun', '하다/Verb', '일상/Noun', '차별/Noun', '구조/Noun', '불평등/Noun', '사실/Noun', '그리다/Verb', '내다/Verb', '조/Noun', '남다/Verb', '작가/Noun', '소설/Noun', '슬하/Noun', '딸/Noun', '서른/Noun', '살/Noun', '전업/Noun', '주부/Noun', '김지영/Noun', '갑자기/Noun', '친정/Noun', '어머니/No

  (Rank 130, 2%) 유아숲 체험장 50개소 설치 등 생애주기별 힐링공원 조성
  (Rank 41, 28%) 걷고 싶은 서울길’ 완성
  (Rank 191, -9%) 생태 친화적 가로수길 ‘우리동네 산책길’ 조성
  (Rank 46, 25%) 생활체육 주말시민리그제 운영
  (Rank 93, 10%) 공원과 산책로 칼로리 게시판 후원합니다
  (Rank 71, 16%) 서울 선데이파크에서 놀면서 살빼요
  (Rank 3, 52%) 건강비법, ‘서울 건강지도’로 확인하세요
  (Rank 135, 1%) 자전거 거치대를 무료로 드립니다
  (Rank 63, 19%) 밤에도 안전한 반짝반짝 빛나는 자전거도로
  (Rank 40, 28%) 시민이 참여하는 사진있는 공원 만들기
  (Rank 173, -6%) 춤추고 노래하는 공원등과 함께해요
  (Rank 46, 25%) 생활체육 주말시민리그제 운영
Precision@10 = 0.1000

Article: 아이 목에 걸린 사탕 간단하게 빼낸 주부...응급처치법은?
동영상 뉴스 [앵커]최근 부산에서 목에 사탕이 걸려 위험에 빠진 아이를 주부가 구한 일이 있었습니다.귀한 목숨을 살린 주부는 오래전 읽었던 육아 지침서를 떠올렸다고 하는데요.김종호 기자가 사연과 함께 목에 음식물이 걸렸을 응급조치법을 소개합니다.[기자]정오 무렵 은행 창구 앞.웃으며 뛰놀던 어린아이가 먹던 사탕이 목에 걸려 숨을 쉬지 못합니다.아이 엄마는 어쩔 몰라 하며 주변에 다급하게 도움을 청합니다.일분일초가 급한 상황, 선뜻 나서는 사람이 없었는데 여성이 아이에게
['아이/Noun', '목/Noun', '걸리다/Verb', '사탕/Noun', '간단하다/Adjective', '빼/Noun', '내다/Verb', '주부/Noun', '응급처치/Noun', '법/Noun', '동영상/Noun', '앵커/Noun', '최근/Noun', '부산/Noun', '목/Noun', '사탕/Noun', '걸리다/Verb', '위험/Noun', '빠지다/Verb

['이마트/Noun', '매장/Noun', '완전/Noun', '철수/Noun', '하다/Verb', '당국/Noun', '점포/Noun', '매각/Noun', '건/Noun', '허가/Noun', '태국/Noun', 'CP/Alpha', '그룹/Noun', '곳/Noun', '매각/Noun', '곳/Noun', '자가/Noun', '점포/Noun', '언제/Noun', '철수/Noun', '가능/Noun', '사실/Noun', '시장/Noun', '완전/Noun', '철수/Noun', '누적/Noun', '적자/Noun', '불다/Verb', '구조조정/Noun', '서울/Noun', '뉴시스/Noun', '최선/Noun', '윤/Noun', '기자/Noun', '이마트/Noun', '중국/Noun', '철수/Noun', '작업/Noun', '사실/Noun', '마무리/Noun', '돼다/Verb', '일/Noun', '관련/Noun', '업계/Noun', '따르다/Verb', '이마트/Noun', '날/Noun', '중국/Noun', '매장/Noun', '곳/Noun', '태국/Noun', 'CP/Alpha', '그룹/Noun', '매각/Noun', '하다/Verb', '과정/Noun', '모두/Noun', '마치다/Verb', '매각/Noun', '건/Noun', '대한/Noun', '중국/Noun', '정부/Noun', '허가/Noun', '나오다/Verb', '서다/Verb', '이마트/Noun', '관계자/Noun', '중국/Noun', '점포/Noun', '매각/Noun', '건/Noun', '대한/Noun', '중국/Noun', '정부/Noun', '허가/Noun', '조금/Noun', '나오다/Verb', '며/Noun', 'CP/Alpha', '그룹/Noun', '간의/Noun', '매각/Noun', '과정/Noun', '모두/Noun']
  Manually tagged promises:
  (None)
Article: 시세 널뛰기 '비트코인

['지하철/Noun', '호선/Noun', '파업/Noun', '예고/Noun', '지옥/Noun', '더/Noun', '하다/Verb', '지옥/Noun', '철/Noun', '되다/Verb', '서울/Noun', '호선/Noun', '노조/Noun', '일/Noun', '일간/Noun', '한시/Noun', '파업/Noun', '조/Noun', '교대/Noun', '조/Noun', '교대/Noun', '전환/Noun', '인/Noun', '역사/Noun', '축소/Noun', '휴식/Noun', '보장/Noun', '요구/Noun', '퇴근/Noun', '시간대/Noun', '평시/Noun', '대비/Noun', '운행/Noun', '률/Noun', '수준/Noun', '감소/Noun', '우려/Noun', '서울/Noun', '호선/Noun', '비상/Noun', '대책/Noun', '운행/Noun', '차질/Noun', '없다/Adjective', '서울/Noun', '지하철/Noun', '호선/Noun', '가양역/Noun', '승강장/Noun', '지하철/Noun', '이용/Noun', '객/Noun', '붐비다/Verb', '있다/Adjective', '사진/Noun', '연합뉴스/Noun', '자료/Noun', '서울/Noun', '지하철/Noun', '호선/Noun', '감사/Noun', '보고서/Noun', '단위/Noun', '억원/Noun', '서울/Noun', '호선/Noun', '노동조합/Noun', '일/Noun', '오전/Noun', '서울/Noun', '중구/Noun', '민주/Noun', '노총/Noun', '월/Noun', '일/Noun', '파업/Noun', '돌입/Noun', '기자회견/Noun', '하다/Verb', '있다/Adjective', '사진/Noun', '연합뉴스/Noun', '이데일리/Noun', '박/Noun', '철근/Noun', '권오석/Noun', '기자/Noun', '서울/Noun', '지하철/Nou

['서울/Noun', '아파트/Noun', '값/Noun', '다시/Noun', '추석/Noun', '연휴/Noun', '후/Noun', '추가/Noun', '대책/Noun', '나오다/Verb', '대책/Noun', '대표/Noun', '되다/Verb', '정부/Noun', '집값/Noun', '안정화/Noun', '정책/Noun', '실효/Noun', '추석/Noun', '연휴/Noun', '이후/Noun', '판가름/Noun', '날/Noun', '것/Noun', '보이다/Verb', '보통/Noun', '추석/Noun', '연휴/Noun', '끝나다/Verb', '가을/Noun', '이사/Noun', '수요/Noun', '늘/Noun', '청약/Noun', '시장/Noun', '활기/Noun', '띠/Noun', '면서/Noun', '집값/Noun', '이다/Verb', '경우/Noun', '많다/Adjective', '올해/Noun', '정부/Noun', '정책/Noun', '변수/Noun', '김현미/Noun', '국토교통부/Noun', '장관/Noun', '지난달/Noun', '일/Noun', '기자간담회/Noun', '국/Noun', '지적/Noun', '과열/Noun', '양상/Noun', '남아/Noun', '있다/Adjective', '장기/Noun', '집값/Noun', '잡/Noun', '있다/Adjective', '보다/Verb', '고/Noun', '말/Noun', '하다/Verb', '집값/Noun', '다시/Noun', '오르다/Verb', '추가/Noun', '대책/Noun', '시장/Noun', '적극/Noun', '개입/Noun', '하다/Verb', '메시지/Noun', '월/Noun', '이후/Noun', '거래/Noun']
  Manually tagged promises:
  (Rank 12, 22%) 안심주택 8만호 공급(2018년)
  (Rank 37, 9%) 관리형 및 공공토지임대형 주택협동조합 육성
  (Rank